In [43]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from datasets import load_dataset

In [44]:
model = TFAutoModel.from_pretrained("bert-base-uncased")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [45]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


In [46]:
inputs = tokenizer(['Hello world', 'Hi how are you shees sheikh ikram'], padding=True, truncation=True,
                  return_tensors='tf')
inputs

{'input_ids': <tf.Tensor: shape=(2, 11), dtype=int32, numpy=
array([[  101,  7592,  2088,   102,     0,     0,     0,     0,     0,
            0,     0],
       [  101,  7632,  2129,  2024,  2017,  2016,  2229, 12840, 20912,
         6444,   102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 11), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 11), dtype=int32, numpy=
array([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [47]:
output =model(inputs)
output

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(2, 11, 768), dtype=float32, numpy=
array([[[-0.16888325,  0.13606349, -0.13940075, ..., -0.6251125 ,
          0.0521726 ,  0.3671454 ],
        [-0.36327422,  0.14121883,  0.8799867 , ...,  0.10433026,
          0.28875732,  0.37267917],
        [-0.6985945 , -0.69879687,  0.06450197, ..., -0.2210373 ,
          0.00986866, -0.5939789 ],
        ...,
        [-0.49337777, -0.23890363,  0.29679617, ..., -0.11194012,
         -0.03332446,  0.00135865],
        [-0.1550966 , -0.09623349,  0.64029384, ..., -0.22854681,
         -0.02242169,  0.19605747],
        [-0.3504956 , -0.09983736,  0.5801807 , ..., -0.18992907,
         -0.0272956 ,  0.19017054]],

       [[-0.36625385,  0.10113718,  0.0441477 , ..., -0.17761455,
          0.8079257 ,  0.15100388],
        [ 0.14330605,  0.31694418,  0.539876  , ..., -0.38345462,
          1.2116374 , -0.47480512],
        [-0.70181537, -0.02192725, -0.04889195, ..

In [48]:
emotions = load_dataset('SetFit/emotion')


Repo card metadata block was not found. Setting CardData to empty.


In [49]:
emotions

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 2000
    })
})

In [50]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [51]:
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)


In [52]:
emotions_encoded


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [53]:
# setting 'input_ids', 'attention_mask', 'token_type_ids', and 'label'
# to the tensorflow format. Now if you access this dataset you will get these
# columns in `tf.Tensor` format

emotions_encoded.set_format('tf', 
                            columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

# setting BATCH_SIZE to 64.
BATCH_SIZE = 64

def order(inp):
    '''
    This function will group all the inputs of BERT
    into a single dictionary and then output it with
    labels.
    '''
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]

# converting train split of `emotions_encoded` to tensorflow format
train_dataset = tf.data.Dataset.from_tensor_slices(emotions_encoded['train'][:])
# set batch_size and shuffle
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
# map the `order` function
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# ... doing the same for test set ...
test_dataset = tf.data.Dataset.from_tensor_slices(emotions_encoded['test'][:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)


In [54]:
inp, out = next(iter(train_dataset)) # a batch from train_dataset
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(64, 87), dtype=int64, numpy=
array([[ 101, 1045, 2514, ...,    0,    0,    0],
       [ 101, 1045, 2001, ...,    0,    0,    0],
       [ 101, 1045, 2071, ...,    0,    0,    0],
       ...,
       [ 101, 1045, 2318, ...,    0,    0,    0],
       [ 101, 1045, 2514, ...,    0,    0,    0],
       [ 101, 1045, 2514, ...,    0,    0,    0]])>, 'attention_mask': <tf.Tensor: shape=(64, 87), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'token_type_ids': <tf.Tensor: shape=(64, 87), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>} 

 tf.Tensor(
[1 4 3 1 0 1 0 1 0 5 1 3 0 4 1 3 1 1 1 0 1 0 3 0 0 1 3 0 0 1 1 

In [55]:
class BERTForClassification(tf.keras.Model):
    
    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [56]:
classifier = BERTForClassification(model, num_classes=6)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [57]:
history = classifier.fit(
    train_dataset,
    epochs=3
)

Epoch 1/3


W0000 00:00:1729797291.474103      98 assert_op.cc:38] Ignoring Assert operator bert_for_classification_1_1/tf_bert_model_2/bert/embeddings/assert_less/Assert/Assert


250/250 ━━━━━━━━━━━━━━━━━━━━ 55s 144ms/step - accuracy: 0.1065 - loss: 1.9167
Epoch 2/3
250/250 ━━━━━━━━━━━━━━━━━━━━ 36s 144ms/step - accuracy: 0.2867 - loss: 1.7168
Epoch 3/3
250/250 ━━━━━━━━━━━━━━━━━━━━ 36s 144ms/step - accuracy: 0.3602 - loss: 1.6326


In [58]:
classifier.evaluate(test_dataset)


W0000 00:00:1729797406.242018      96 assert_op.cc:38] Ignoring Assert operator bert_for_classification_1_1/tf_bert_model_2/bert/embeddings/assert_less/Assert/Assert


31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.3588 - loss: 1.6141

W0000 00:00:1729797411.659490      99 assert_op.cc:38] Ignoring Assert operator bert_for_classification_1_1/tf_bert_model_2/bert/embeddings/assert_less/Assert/Assert


32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 169ms/step - accuracy: 0.3603 - loss: 1.6131


[1.5969398021697998, 0.38350000977516174]

In [59]:
def predict_emotion(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='tf', max_length=128)
    predictions = classifier(inputs)  # Using the trained classifier model
    predicted_classes = tf.argmax(predictions, axis=1).numpy()
    
    # Emotion labels corresponding to the class indices
    emotion_labels = ['anger', 'fear', 'joy', 'love', 'sadness', 'surprise']
    
    return [emotion_labels[pred] for pred in predicted_classes]

In [60]:
# Function to predict emotion from the model


# Sample texts to predict emotions
sample_texts = [
    "I just got a promotion at work! I can't believe how happy I am!",
    "I lost my best friend recently, and I feel so lonely without them.",
    "I am really frustrated with the way they handled the situation.",
    "I can't stop thinking about the dark shadows outside my window.",
    "Every moment spent with you feels like a dream come true.",
    "I was shocked to find out I won the lottery!"
]

# Get predictions
predictions = predict_emotion(sample_texts)
print(f'Predicted emotions: {predictions}')


ValueError: Only input tensors may be passed as positional arguments. The following argument value should be passed as a keyword argument: {'input_ids': <tf.Tensor: shape=(6, 20), dtype=int32, numpy=
array([[  101,  1045,  2074,  2288,  1037,  4712,  2012,  2147,   999,
         1045,  2064,  1005,  1056,  2903,  2129,  3407,  1045,  2572,
          999,   102],
       [  101,  1045,  2439,  2026,  2190,  2767,  3728,  1010,  1998,
         1045,  2514,  2061,  9479,  2302,  2068,  1012,   102,     0,
            0,     0],
       [  101,  1045,  2572,  2428, 10206,  2007,  1996,  2126,  2027,
         8971,  1996,  3663,  1012,   102,     0,     0,     0,     0,
            0,     0],
       [  101,  1045,  2064,  1005,  1056,  2644,  3241,  2055,  1996,
         2601,  6281,  2648,  2026,  3332,  1012,   102,     0,     0,
            0,     0],
       [  101,  2296,  2617,  2985,  2007,  2017,  5683,  2066,  1037,
         3959,  2272,  2995,  1012,   102,     0,     0,     0,     0

In [61]:
def predict_emotion(texts, emotion_labels=['anger', 'fear', 'joy', 'love', 'sadness', 'surprise']):
    # Check if texts are provided
    if not texts:
        raise ValueError("Input texts cannot be empty.")
    
    # Tokenization
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='tf', max_length=128)
    
    # Model Prediction (unpacking inputs as keyword arguments)
    predictions = classifier(**inputs)  # Unpack the inputs dictionary
    predicted_classes = tf.argmax(predictions, axis=1).numpy()
    
    # Map to emotion labels
    return [emotion_labels[pred] for pred in predicted_classes]

# Sample usage
sample_texts = [
    "I just got a promotion at work! I can't believe how happy I am!",
    "I lost my best friend recently, and I feel so lonely without them.",
    "I was shocked to find out I won the lottery!"
]

# Get predictions
predictions = predict_emotion(sample_texts)
print(f'Predicted emotions: {predictions}')


TypeError: missing a required argument: 'inputs'